In [1]:
import logging
import math
import random
import numpy as np
import time
import torch 
import torch.nn as nn
from torch import optim
from torch.nn import utils
import matplotlib.pyplot as plt

!pip install ipywidgets
from ipywidgets import IntProgress
from IPython.display import display
from matplotlib import pyplot as plt, rc
from matplotlib.animation import FuncAnimation, PillowWriter
rc('animation', html='jshtml')
!pip install jupyterthemes
from jupyterthemes import jtplot
jtplot.style(theme='grade3', context='notebook', ticks=True, grid=False)

logger = logging.getLogger(__name__)

                                              0.0/7.0 MB ? eta -:--:--
                                              0.2/7.0 MB 3.5 MB/s eta 0:00:02
     ---                                      0.7/7.0 MB 7.0 MB/s eta 0:00:01
     -----                                    1.0/7.0 MB 7.0 MB/s eta 0:00:01
     --------                                 1.5/7.0 MB 7.8 MB/s eta 0:00:01
     ---------                                1.6/7.0 MB 8.5 MB/s eta 0:00:01
     -----------                              2.0/7.0 MB 7.0 MB/s eta 0:00:01
     --------------                           2.6/7.0 MB 7.9 MB/s eta 0:00:01
     ----------------                         3.0/7.0 MB 7.9 MB/s eta 0:00:01
     ----------------                         3.0/7.0 MB 7.9 MB/s eta 0:00:01
     -------------------                      3.4/7.0 MB 7.5 MB/s eta 0:00:01
     -------------------                      3.4/7.0 MB 7.5 MB/s eta 0:00:01
     ---------------------                    3.7/7.0 MB 6.7 MB/s eta 0

ModuleNotFoundError: No module named 'jupyterthemes'

In [ ]:
class Dynamics(nn.Module):

    def __init__(self):
        super(Dynamics, self).__init__()

    def forward(self, state, action):
        """
        action[0] = ax
        action[1] = ay
        action[3] = az
        g = 10
        state[0] = r
        state[1] = r_dot
        state[2] = theta
        state[3] = theta_dot
        state[4] = z
        state[5] = z_dot
        
      
        """
        # Apply gravity
        # Note: Here gravity is used to change velocity which is the second element of the state vector
        # Normally, we would do x[1] = x[1] + gravity * delta_time
        # but this is not allowed in PyTorch since it overwrites one variable (x[1]) that is part of the computational graph to be differentiated.
        # Therefore, I define a tensor dx = [0., gravity * delta_time], and do x = x + dx. This is allowed.

        delta_state_gravity = torch.tensor([[0., 0.,-GRAVITY_ACCEL_X * FRAME_TIME, -GRAVITY_ACCEL_Y * FRAME_TIME, 0.]])

        state_tensor = torch.zeros((1, 5))
        state_tensor[0, 3] = torch.cos(state[0, 4])
        state_tensor[0, 2] = torch.sin(state[0, 4])
    
        delta_state = BOOST_ACCEL * FRAME_TIME * torch.mul(state_tensor, action[0, 0].reshape(-1, 1))

        # Theta
        delta_state_theta = FRAME_TIME * OMEGA_RATE * torch.mul(torch.tensor([0., 0., 0., 0, 1.]),action[0, 1].reshape(-1, 1)) 

        # Update state
        step_mat = torch.tensor([[1., 0.,FRAME_TIME, 0., 0.],
                                 [0., 1., 0., FRAME_TIME, 0.],
                                 [0., 0., 1., 0., 0.],
                                 [0., 0., 0., 1., 0.],
                                 [0., 0., 0., 0., 1.]])
        
        shift_mat = torch.tensor([[0., 0.,FRAME_TIME, 0., 0.],
                                 [0., 0., 0., FRAME_TIME, 0.],
                                 [0., 0., 0., 0., 0.],
                                 [0., 0., 0., 0., 0.],
                                 [0., 0., 0., 0., 0.]])

        state = torch.matmul(step_mat, state.T) + torch.matmul(shift_mat, delta_state.T)  * 0.5 + torch.matmul(shift_mat, delta_state_gravity.T)  * 0.5
        state = state.T

        state = state + delta_state  + delta_state_gravity + delta_state_theta

        return state